# Benchmark pyfoamalgo - Image Processing

Author: Jun Zhu

In [ ]:
import random
import numpy as np

import pyfoamalgo
print("pyfoamalgo version: ", pyfoamalgo.__version__)

import multiprocessing as mp
print("Number of cores: ", mp.cpu_count())

from pyfoamalgo import mask_image_data, correct_image_data

---
## Initialize image data

In [ ]:
NUM_IMAGES = 128
IMAGE_SHAPE = (1200, 1124)
DTYPE = np.float32

In [ ]:
def generate_image_array(n, with_nan=True):
    data = np.random.randn(n, *IMAGE_SHAPE).astype(DTYPE)
    if with_nan:
        data[:, ::2, ::2] = np.nan
    return data


def generate_image(with_nan=True):
    data = np.random.randn(*IMAGE_SHAPE).astype(DTYPE)
    if with_nan:
        data[::2, ::2] = np.nan
    return data


image_mask = np.ones(IMAGE_SHAPE, dtype=bool)
image_mask[::3, ::3] = np.nan

gains = np.random.randn(NUM_IMAGES, *IMAGE_SHAPE).astype(DTYPE) / 10.
offsets = np.random.randn(NUM_IMAGES, *IMAGE_SHAPE).astype(DTYPE)

---
## Masking image(s)

### Threshold mask (image array)

In [ ]:
imgs = generate_image_array(NUM_IMAGES)
%timeit imgs[(imgs > 1) | (imgs < -1)] = np.nan

In [ ]:
imgs = generate_image_array(NUM_IMAGES)
%timeit mask_image_data(imgs, threshold_mask=(-1, 1))

### Threshold mask (single image)

In [ ]:
img = generate_image()
%timeit img[(img > 1) | (img < -1)] = np.nan

In [ ]:
img = generate_image()
%timeit mask_image_data(img, threshold_mask=(-1, 1))

### Image mask (image array)

In [ ]:
imgs = generate_image_array(NUM_IMAGES)
%timeit imgs[:, image_mask] = np.nan

In [ ]:
imgs = generate_image_array(NUM_IMAGES)
%timeit mask_image_data(imgs, image_mask=image_mask)

### Image mask (single image)

In [ ]:
img = generate_image()
%timeit img[image_mask] = np.nan

In [ ]:
img = generate_image()
%timeit mask_image_data(img, image_mask=image_mask)

### Threshold mask + Image mask (image array)

In [ ]:
imgs = generate_image_array(NUM_IMAGES)
%timeit imgs[(image_mask) | (imgs > 1) | (imgs < -1)] = np.nan

In [ ]:
imgs = generate_image_array(NUM_IMAGES)
%timeit mask_image_data(imgs, image_mask=image_mask, threshold_mask=(-1, 1))

### Threshold mask + Image mask (single image)

In [ ]:
img = generate_image()
%timeit img[(image_mask) | (img > 1) | (img < -1)] = np.nan

In [ ]:
img = generate_image()
%timeit mask_image_data(img, image_mask=image_mask, threshold_mask=(-1, 1))

---
## Correcting (Calibrating) detector image(s)

In [ ]:
def correct_gain_py(a, b):
    a *= b
    
def correct_offset_py(a, b):
    a -= b
    
def correct_py(a, b, c):
    a = (a - b) * c

### correcting gain (image array)

In [ ]:
imgs = generate_image_array(NUM_IMAGES)
%timeit correct_gain_py(imgs, gains)

In [ ]:
imgs = generate_image_array(NUM_IMAGES)
%timeit correct_image_data(imgs, gain=gains)

### correcting gain (image array)

In [ ]:
img = generate_image()
%timeit correct_gain_py(img, gains[0])

In [ ]:
img = generate_image()
%timeit correct_image_data(img, gain=gains[0])

### correcting offset (image array)

In [ ]:
imgs = generate_image_array(NUM_IMAGES)
%timeit correct_offset_py(imgs, offsets)

In [ ]:
imgs = generate_image_array(NUM_IMAGES)
%timeit correct_image_data(imgs, offset=offsets)

### correcting offset (single image)

In [ ]:
img = generate_image()
%timeit correct_offset_py(img, offsets[0])

In [ ]:
img = generate_image()
%timeit correct_image_data(img, offset=offsets[0])

### correcting offset&gain (image array)

In [ ]:
imgs = generate_image_array(NUM_IMAGES)
%timeit correct_py(imgs, offsets, gains)

In [ ]:
imgs = generate_image_array(NUM_IMAGES)
%timeit correct_image_data(imgs, offset=offsets, gain=gains)

### correcting gain&offset (sing image)

In [ ]:
img = generate_image()
%timeit correct_py(img, offsets[0], gains[0])

In [ ]:
img = generate_image()
%timeit correct_image_data(img, offset=offsets[0], gain=gains[0])